In [44]:
import cracknuts as cn

In [45]:
g1 = cn.cracker_g1('192.168.0.10')
g1.connect(force_update_bin=True, force_write_default_config=True)

In [46]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(cracker):
    # print(cracker.get_current_config())
    # g1.nut_clock_freq('8m')
    # g1.nut_clock_enable()
    # g1.nut_voltage_enable()
    
    cracker.nut_clock_freq('8M')
    cracker.nut_clock_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_voltage_enable()
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    # cracker.osc_analog_enable('A')
    # cracker.osc_analog_gain('A', 1)
    # cracker.osc_analog_disable('B')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(
        baudrate=serial.Baudrate.BAUDRATE_115200, 
        bytesize=serial.Bytesize.EIGHTBITS, 
        parity=serial.Parity.PARITY_NONE, 
        stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(cracker, count):
    # plaintext_data = random.randbytes(aes_data_len)
    plaintext_data = bytes.fromhex('AA BB CC DD EE FF 00 11 22 33 44 55 66 77 88 99')
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    print(ret.hex(' ') if ret else 'None')
    return {
        "plaintext": plaintext_data,
        "ciphertext": b'',
        "key": bytes.fromhex(aes_key)
    }


acq = cn.simple_glitch_acq(g1, init, do)

p = cn.show_panel(acq)
p

In [ ]:
g1.nut_clock_enable()

In [21]:
g1.nut_voltage_disable()

(0, None)

In [ ]:
g1.nut_voltage_enable()

In [42]:
g1 = cn.cracker_g1('192.168.0.10')
g1.connect(force_update_bin=True, force_write_default_config=True)

In [43]:
g1.nut_clock_freq('8m')
g1.nut_clock_enable()
g1.nut_voltage_enable()

(0, None)

In [ ]:
g1.nut_clock_freq('8m')

In [ ]:
g1.register_read(base_address=0x43c10000, offset=0x1C4)

In [36]:
import random
import time
from cracknuts.cracker import serial

g1.nut_clock_freq('8M')
g1.nut_clock_enable()
g1.nut_voltage(3.3)
g1.nut_voltage_enable()
g1.uart_io_enable()
g1.osc_sample_clock('48m')
g1.osc_sample_length(sample_length)
g1.osc_trigger_source('N')
# cracker.osc_analog_enable('A')
# cracker.osc_analog_gain('A', 1)
# cracker.osc_analog_disable('B')
g1.osc_analog_gain('B', 10)
g1.osc_trigger_level(0)
g1.osc_trigger_mode('E')
g1.osc_trigger_edge('U')
g1.uart_config(
    baudrate=serial.Baudrate.BAUDRATE_115200, 
    bytesize=serial.Bytesize.EIGHTBITS, 
    parity=serial.Parity.PARITY_NONE, 
    stopbits=serial.Stopbits.STOPBITS_ONE)

(0, None)

In [23]:
cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

cmd = cmd_set_aes_enc_key + aes_key
g1.uart_transmit_receive(cmd, timeout=1000, rx_count=6)


[WARNING] 2025-12-31 15:49:05,256 cracker_basic.send_and_receive:526 Command 0x0263 with payload 00 06 00 00 00 03 e8 01 00 00 00 00 00 00 10 11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff received a non-OK response with status code 0x0002, and payload [ ].


(2, b'\x00')

In [9]:
plaintext_data = bytes.fromhex('AA BB CC DD EE FF 00 11 22 33 44 55 66 77 88 99')
tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
status, ret = g1.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
print(ret.hex(' ') if ret else 'None')

00 00 00 00 00 10 c7 ee 06 6c 38 bb 1c 07 97 61 9c ad 8a 2b ca 6a


In [40]:
# c = p.get_cracker_panel_config()
p.write_config_to_cracker()

In [47]:
p.get_cracker_panel_config()

Config(osc_channel_0_enable: False, osc_channel_1_enable: True, osc_channel_0_gain: 10, osc_channel_1_gain: 10, osc_sample_length: 1024, osc_sample_delay: 0, osc_sample_clock: 48000, osc_sample_phase: 0, osc_trigger_source: 0, osc_trigger_mode: 0, osc_trigger_edge: 0, osc_trigger_edge_level: 1, nut_enable: False, nut_voltage: 3.3, nut_clock_enable: False, nut_clock: 8000.0, nut_timeout: 0, nut_reset_io_enable: False, nut_uart_enable: False, nut_uart_baudrate: Baudrate.BAUDRATE_115200, nut_uart_bytesize: Bytesize.EIGHTBITS, nut_uart_parity: Parity.PARITY_NONE, nut_uart_stopbits: Stopbits.STOPBITS_ONE, nut_spi_enable: False, nut_spi_speed: 5000.0, nut_spi_cpol: SpiCpol.SPI_CPOL_LOW, nut_spi_cpha: SpiCpha.SPI_CPHA_LOW, nut_spi_csn_auto: True, nut_spi_csn_delay: True, nut_i2c_enable: False, nut_i2c_dev_addr: 0, nut_i2c_speed: I2cSpeed.STANDARD_100K, nut_i2c_stretch_enable: False, glitch_vcc_normal: 3.3, glitch_vcc_config_wait: 2667577344, glitch_vcc_config_level: 0.0, glitch_vcc_config_cou

In [48]:
g1.get_current_config()

Config(osc_channel_0_enable: False, osc_channel_1_enable: True, osc_channel_0_gain: 10, osc_channel_1_gain: 10, osc_sample_length: 1024, osc_sample_delay: 0, osc_sample_clock: 48000, osc_sample_phase: 0, osc_trigger_source: 0, osc_trigger_mode: 0, osc_trigger_edge: 0, osc_trigger_edge_level: 1, nut_enable: False, nut_voltage: 3.3, nut_clock_enable: False, nut_clock: 8000, nut_timeout: None, nut_reset_io_enable: False, nut_uart_enable: False, nut_uart_baudrate: Baudrate.BAUDRATE_115200, nut_uart_bytesize: Bytesize.EIGHTBITS, nut_uart_parity: Parity.PARITY_NONE, nut_uart_stopbits: Stopbits.STOPBITS_ONE, nut_spi_enable: False, nut_spi_speed: 5000, nut_spi_cpol: SpiCpol.SPI_CPOL_LOW, nut_spi_cpha: SpiCpha.SPI_CPHA_LOW, nut_spi_csn_auto: True, nut_spi_csn_delay: True, nut_i2c_enable: False, nut_i2c_dev_addr: 0, nut_i2c_speed: I2cSpeed.STANDARD_100K, nut_i2c_stretch_enable: False, glitch_vcc_normal: 3.3, glitch_vcc_config_wait: 2667577344, glitch_vcc_config_level: 0.0, glitch_vcc_config_coun